In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Input, MaxPooling1D, GlobalAveragePooling1D, MultiHeadAttention, Dropout, LayerNormalization

# Завантаження даних
data = pd.read_csv('electricity_dah_prices.csv')
data['Datetime'] = pd.to_datetime(data['date'] + ' ' + data['hour'].str[:5])
data.set_index('Datetime', inplace=True)
data.drop(columns=['date', 'hour'], inplace=True)

# Вибір цільової змінної, наприклад 'france'
prices = data['france'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices)

# Підготовка вхідних даних
def create_dataset(dataset, window_size=24):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size)]
        dataX.append(a)
        dataY.append(dataset[i + window_size])
    return np.array(dataX), np.array(dataY)

X, y = create_dataset(scaled_prices)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Побудова CNN
model_cnn = Sequential([
    Input(shape=(24, 1)),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(2),
    Conv1D(128, kernel_size=3, activation='relu'),
    GlobalAveragePooling1D(),
    Dense(1)
])
model_cnn.compile(optimizer='adam', loss='mean_squared_error')
model_cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

### Побудова трансформера
input_layer = Input(shape=(24, 1))
attention_layer = MultiHeadAttention(num_heads=2, key_dim=24)(input_layer, input_layer)
add_layer = LayerNormalization(epsilon=1e-6)(input_layer + attention_layer)
output_layer = Dense(1)(add_layer)

model_transformer = Model(inputs=input_layer, outputs=output_layer)
model_transformer.compile(optimizer='adam', loss='mean_squared_error')
model_transformer.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: nan - val_loss: nan
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: nan - val_loss: nan
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 9/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 10/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout

# Параметри моделі
num_features = 1  # тому що ми використовуємо один стовпець даних
window_size = 24  # використовуємо дані за останні 24 години для прогнозу

# Функція для створення одного трансформер-блоку
def transformer_block(inputs):
    # Самоувага
    attn_output = MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
    attn_output = Dropout(0.1)(attn_output)
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # Повнозв'язний шар
    ffn_output = Dense(64, activation="relu")(out1)
    ffn_output = Dropout(0.1)(ffn_output)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Вхідні дані
inputs = Input(shape=(window_size, num_features))

# Трансформер-блок
x = transformer_block(inputs)

# Вихідний шар
outputs = Dense(1)(x)

# Створення моделі
model_transformer = Model(inputs=inputs, outputs=outputs)

# Компіляція моделі
model_transformer.compile(optimizer='adam', loss='mse')

# Вивід структури моделі
model_transformer.summary()

# Навчання моделі
model_transformer.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)    │ (None, 24, 1)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention_2        │ (None, 24, 1)             │              29 │ input_layer_4[0][0],       │
│ (MultiHeadAttention)          │                           │                 │ input_layer_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 24, 1)             │               0 │ multi_head_attention_2[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_2 (Add)                   │ (None, 24, 1)             │               0 │ input_layer_4[0][0],       │
│                               │                           │                 │ dropout_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_2         │ (None, 24, 1)             │               2 │ add_2[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 24, 64)            │             128 │ layer_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_4 (Dropout)           │ (None, 24, 64)            │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_3 (Add)                   │ (None, 24, 64)            │               0 │ layer_normalization_2[0][… │
│                               │                           │                 │ dropout_4[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_3         │ (None, 24, 64)            │             128 │ add_3[0][0]                │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 24, 1)             │              65 │ layer_normalization_3[0][… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 352 (1.38 KB)

 Trainable params: 352 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: nan - val_loss: nan
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - val_loss: nan
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - val_loss: nan
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - val_loss: nan
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - val_loss: nan
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 9/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
Epoch 10/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: nan - val_loss: nan
